In [ ]:
using Revise
import TensorCrossInterpolation as TCI
using Plots
using BenchmarkTools
using Test

In [ ]:
localdims = [2, 2, 2, 2, 2]
localset = [collect(1:d) for d in localdims]
leftindexset = [[1,1] for _ in 1:100]
rightindexset = [[1,1] for _ in 1:100]

module My
import TensorCrossInterpolation as TCI
struct TestFunction <: TCI.BatchEvaluator{Float64}
    localset::Vector{Vector{Int}}
    function TestFunction(localset)
        new(localset)
    end
end

(obj::TestFunction)(indexset)::Float64 = sum(indexset)
function (obj::TestFunction)(leftindexset, rightindexset, ::Val{M})::Array{Float64,M+2} where {M}
    nl = length(first(leftindexset))
    result = [sum(vcat(l, collect(c), r)) for l in leftindexset, c in Iterators.product((1:length(d) for d in obj.localset[nl+1:nl+M])...), r in rightindexset]
    return reshape(result, length(leftindexset), length.(obj.localset[nl+1:nl+M])..., length(rightindexset))
end

end

f = TCI.CachedFunction{Float64}(My.TestFunction(localset), localdims)
@assert TCI.isbatchevaluable(f)
result = TCI._batchevaluate_dispatch(Float64, f, localset, leftindexset, rightindexset, Val(1))
ref = [sum(vcat(l, c, r)) for l in leftindexset, c in localset[3], r in rightindexset]

result ≈ ref

In [ ]:
@code_warntype TCI._batchevaluate_dispatch(Float64, f, localset, leftindexset, rightindexset, Val(1))

In [ ]:
function func(leftindexset, rightindexset, dims)
    nl = length(first(leftindexset))
    nr = length(first(rightindexset))
    L = length(dims)

    r = 0.0
    for i in Iterators.product(leftindexset, (1:dims[l] for l in nl+1:L-nr)..., rightindexset)
        r += 1.0
    end
    #return V[
        1.0
        #for i in Iterators.product(leftindexset, (1:dims[l] for l in nl+1:L-nr)..., rightindexset)
    #]
end

In [ ]:
localdims = [2, 2, 2, 2, 2]
x = [[1,1] for _ in 1:100]
y = [[1,1] for _ in 1:100]
@code_warntype func(x, y, localdims)

In [ ]:
@code_warntype func(x, y, localdims)

In [ ]:
function f()
    return 2
end

function has_zero_arg_method(f)
    methods_list = methodswith(typeof(f))
    for m in methods_list
        if m.nargs == 1  # 引数が自身の関数オブジェクトのみ（つまり引数なし）
            return true
        end
    end
    return false
end

@benchmark has_zero_arg_method(f)

In [ ]:
function run(N, maxiter)
    firstpivot = ones(Int, N)
    qtt, ranks, errors = TCI.crossinterpolate(
        Float64, x->randn(), fill(2, N), firstpivot; tolerance=0.0, maxiter=maxiter, verbosity=1
    )
end

In [ ]:
# Compile
run(10, 10)

In [ ]:
Ds = [20, 40, 80, 160, 240, 400, 800, 1000]

timings = Float64[]
for maxiter in Ds
    t1 = time_ns()
    @time run(20, maxiter)
    t2 = time_ns()
    push!(timings, (t2-t1)*1e-9)
end

In [ ]:
p = plot(xaxis=:log, yaxis=:log)
plot!(p, Ds, 1e-3*Ds.^2, label="D^2")
plot!(p, Ds, 1e-5*Ds.^3, label="D^3")
plot!(p, Ds, timings, marker=:cross)
